# 02 - Experimentation - Vertex AI

This notebook covers the following steps:

1. Build the Docker container image.
2. Submit a `Vertex AI` custom job to prepare the data.
3. Submit a `Vertex AI` custom job to train and export the model.
4. Upload the exported model as a Vertex AI model resource.

## Setup

In [ ]:
import os
import logging
import time
from datetime import datetime

import tensorflow as tf
from google.cloud import aiplatform as vertex_ai

logging.getLogger().setLevel(logging.INFO)
tf.get_logger().setLevel('INFO')

print(f"TensorFlow: {tf.__version__}")

In [ ]:
PROJECT = 'merlin-on-gcp'
REGION = 'us-central1'
BUCKET = 'merlin-on-gcp'
VERTEX_SERVICE_ACCOUNT = f'vertex-sa-mlops@{PROJECT}.iam.gserviceaccount.com'

MOVIES_DATASET_DISPLAY_NAME = 'movielens25m-movies'
RATINGS_DATASET_DISPLAY_NAME = 'movielens25m-ratings'

MODEL_DISPLAY_NAME = f'movielens25m-recommender'

WORKSPACE = f"gs://{BUCKET}/movielens25m"
EXPERIMENT_ARTIFACTS_DIR = os.path.join(WORKSPACE, 'experiments')

TENSORBOARD_DISPLAY_NAME = f'tb-{PROJECT}'
EXPERIMENT_NAME = f'{MODEL_DISPLAY_NAME}-experiment'

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=BUCKET,
    experiment=EXPERIMENT_NAME,
)

## Create Vertex TensorBoard Instance

In [ ]:
!gcloud beta ai tensorboards create --display-name={TENSORBOARD_DISPLAY_NAME} \
  --project={PROJECT} --region={REGION}

In [ ]:
TENSORBOARD_RESOURCE_NAME = "projects/659831510405/locations/us-central1/tensorboards/4450717516120981504"

## Initialize Vertex AI Experiment

In [ ]:
REMOVE_EXPERIMENT_ARTIFACTS = False
if tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR) and REMOVE_EXPERIMENT_ARTIFACTS:
    print("Removing previous experiment artifacts...")
    tf.io.gfile.rmtree(EXPERIMENT_ARTIFACTS_DIR)

if not tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR):
    print("Creating new experiment artifacts directory...")
    tf.io.gfile.mkdir(EXPERIMENT_ARTIFACTS_DIR)

print("Workspace is ready.")

run_id = f"run-local-{datetime.now().strftime('%Y%m%d%H%M%S')}"
vertex_ai.start_run(run_id)

EXPERIMENT_RUN_DIR = os.path.join(EXPERIMENT_ARTIFACTS_DIR, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

In [ ]:
def get_dataset_gcs_location(dataset_display_name):
    datasets = vertex_ai.TabularDataset.list()
    dataset = None
    for entry in datasets:
        if entry.display_name == dataset_display_name:
            dataset = entry
            break

    if not dataset:
        raise ValueError(f"Dataset with display name {dataset_display_name} does not exist!")
    
    return dataset.gca_resource.metadata['inputConfig']['gcsSource']['uri'][0]

In [ ]:
movies_csv_dataset_location = get_dataset_gcs_location(MOVIES_DATASET_DISPLAY_NAME)
ratings_csv_dataset_location = get_dataset_gcs_location(RATINGS_DATASET_DISPLAY_NAME)
print("Movies CSV dataset location:", ratings_csv_dataset_location)
print("Ratings CSV dataset location:", ratings_csv_dataset_location)

## 3. Build training container image

In [ ]:
IMAGE_NAME="movielens-nvt0.3-tf2.4"
IMAGE_URI=f"gcr.io/{PROJECT}/{IMAGE_NAME}"
print(IMAGE_URI)

In [ ]:
! gcloud builds submit --tag {IMAGE_URI} . --timeout=45m --machine-type=e2-highcpu-8

## 4. Submit Vertex AI Custom Job for ETL

### Prepare worker pool specification

In [ ]:
ETL_OUTPUT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'etl_output')

In [ ]:
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_V100",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": ["python", "-m", "src.data_preprocessing.task"],
            "args": [
                f'--movies-csv-dataset-location={movies_csv_dataset_location}',
                f'--ratings-csv-dataset-location={ratings_csv_dataset_location}',
                f'--etl-output-dir={ETL_OUTPUT_DIR}',
            ],
        },
    }
]


### Submit and monitor the job

In [ ]:
job_name = "movielens-nvt-etl-{}".format(time.strftime("%Y%m%d_%H%M%S"))

job = vertex_ai.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=os.path.join(WORKSPACE, 'jobs')
)

job.run(
    sync=True, 
    service_account=VERTEX_SERVICE_ACCOUNT,
    tensorboard=TENSORBOARD_RESOURCE_NAME
)

## 5. Submit Vertex AI Custom Job for Model Training

### Prepare worker pool specification

In [ ]:
TRANSFORM_TRAIN_DATA = os.path.join(ETL_OUTPUT_DIR, 'transformed_data/train/*.parquet')
TRANSFORM_EVAL_DATA = os.path.join(ETL_OUTPUT_DIR, 'transformed_data/test/*.parquet')
TRANSFORM_WORKFLOW_DIR = os.path.join(ETL_OUTPUT_DIR, 'transform_workflow')
NUM_EPOCHS = 1
LEARNING_RATE = 0.001

In [ ]:
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_V100",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": ["python", "-m", "src.model_training.task"],
            "args": [
                f'--train-data-file-pattern={TRANSFORM_TRAIN_DATA}',
                f'--eval-data-file-pattern={TRANSFORM_EVAL_DATA}',
                f'--nvt-workflow-dir={TRANSFORM_WORKFLOW_DIR}',
                f'--num-epochs={NUM_EPOCHS}',
                f'--learning-rate={LEARNING_RATE}',
            ],
        },
    }
]

### Submit and monitor the job

In [ ]:
job_name = "movielens-tf-training-{}".format(time.strftime("%Y%m%d_%H%M%S"))

job = vertex_ai.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)

job.run(
    sync=True, 
    service_account=VERTEX_SERVICE_ACCOUNT,
    tensorboard=TENSORBOARD_RESOURCE_NAME
)

## 6. Upload the model Vertex AI

In [ ]:
TRITON_SERVER_IMAGE = f"gcr.io/{PROJECT}/triton-{MODEL_DISPLAY_NAME}:latest"

In [ ]:
vertex_ai.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=EXPORT_DIR,
    serving_container_image_uri=TRITON_SERVER_IMAGE
)